In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.optim import Adam
import timm
from tqdm import tqdm
import argparse
import os


def laguerre_polynomial(x, degree):
    if degree == 0:
        return torch.ones_like(x)
    elif degree == 1:
        return 1 - x

    L_prev_prev = torch.ones_like(x)
    L_prev = 1 - x
    for n in range(2, degree + 1):
        L = ((2 * n - 1 - x) * L_prev - (n - 1) * L_prev_prev) / n
        L_prev_prev, L_prev = L_prev, L
    return L

class KANLayer(nn.Module):
    def __init__(self, input_dim, output_dim, degree=3, grid_size=4, use_resid=False, resid=nn.SiLU):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.degree = degree
        self.grid_size = grid_size
        self.use_resid = use_resid

        self.grid_centers = nn.Parameter(torch.linspace(0, 1, grid_size).view(1, 1, 1, grid_size))
        self.grid_scales = nn.Parameter(torch.ones(input_dim, grid_size))
        self.coefficients = nn.Parameter(torch.randn(input_dim, grid_size, output_dim))

        if self.use_resid:
            self.resid_act = resid()
            self.resid_linear = nn.Linear(input_dim, output_dim)

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.constant_(self.grid_scales, 1.0)

        nn.init.normal_(self.coefficients, std=1 / (self.grid_size ** 0.5))
        if self.use_resid:
            nn.init.kaiming_normal_(self.resid_linear.weight)
            nn.init.zeros_(self.resid_linear.bias)

    def kan_transform(self, x):
        x_transformed = (x.unsqueeze(-1) - self.grid_centers) * self.grid_scales.abs()
        x_transformed = torch.clamp(x_transformed, -1, 1)  # Clamp to avoid large values

        basis = laguerre_polynomial(x_transformed, self.degree)
        out = torch.einsum('bldg,dgo->blo', basis, self.coefficients)
        return out

    def forward(self, x):
        out = self.kan_transform(x)
        if self.use_resid:
            res = self.resid_linear(self.resid_act(x))
            out = out + res
        return out



class ViTWithKAN(nn.Module):
    def __init__(self, num_classes, hidden_dims=[256, 128], kan_degree=3, grid_size=4, vit_model_name='vit_base_patch16_224'):
        super().__init__()

        # Load pretrained ViT from timm
        self.backbone = timm.create_model(vit_model_name, pretrained=True)
        self.backbone.head = nn.Identity()  # Remove the classification head

        in_features = self.backbone.num_features  # Typically 768 for vit_base_patch16_224

        layers = []
        input_dim = in_features
        for hidden_dim in hidden_dims:
            layers.append(KANLayer(input_dim, hidden_dim, degree=kan_degree, grid_size=grid_size, use_resid=True))
            input_dim = hidden_dim

        layers.append(KANLayer(input_dim, num_classes, degree=kan_degree, grid_size=grid_size, use_resid=True))
        self.kan_head = nn.Sequential(*layers)

    def forward(self, x):
        x = self.backbone(x)  # [B, D] output of CLS token
        x = x.unsqueeze(1)    # [B, 1, D]
        x = self.kan_head(x)  # [B, 1, num_classes]
        return x.squeeze(1)   # [B, num_classes]



In [2]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for images, labels in tqdm(loader, desc="Train", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        total_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return total_loss / total, correct / total

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    for images, labels in tqdm(loader, desc="Eval", leave=False):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        total_loss += loss.item() * labels.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return total_loss / total, correct / total





In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.optim import Adam

# Assuming ViTWithKAN is defined elsewhere and imported

def main(args):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # ViT expects 224x224 inputs and ImageNet normalization
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.RandomCrop(224, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406),  # ImageNet mean
                             (0.229, 0.224, 0.225))  # ImageNet std
    ])

    # Load CIFAR-10 or CIFAR-100
    if args.dataset == 'cifar10':
        trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
        num_classes = 10
    else:
        trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
        testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
        num_classes = 100

    train_loader = DataLoader(trainset, batch_size=args.batch_size, shuffle=True, num_workers=4)
    test_loader = DataLoader(testset, batch_size=args.batch_size, shuffle=False, num_workers=4)


    model = ViTWithKAN(num_classes=num_classes,
                   hidden_dims=[256, 128],
                   kan_degree=args.degree,
                   grid_size=args.grid_size,
                   vit_model_name='vit_tiny_patch16_224').to(device)


    optimizer = Adam(model.parameters(), lr=args.lr)
    criterion = nn.CrossEntropyLoss()

    best_acc = 0
    for epoch in range(args.epochs):
        print(f"\nEpoch {epoch + 1}/{args.epochs}")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        test_loss, test_acc = evaluate(model, test_loader, criterion, device)

        print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
        print(f"Test  Loss: {test_loss:.4f}, Acc: {test_acc:.4f}")

        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), f'best_model_{args.dataset}_vit.pth')

    print(f"\nBest Accuracy on {args.dataset}: {best_acc:.4f}")


In [4]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', type=str, default='cifar10', choices=['cifar10', 'cifar100'])
    parser.add_argument('--epochs', type=int, default=100)
    parser.add_argument('--batch_size', type=int, default=128)
    parser.add_argument('--lr', type=float, default=1e-3)
    parser.add_argument('--degree', type=int, default=3)
    parser.add_argument('--grid_size', type=int, default=4)


    args, _ = parser.parse_known_args()
    return args

if __name__ == '__main__':
    args = get_args()
    main(args)


100%|██████████| 170M/170M [00:03<00:00, 48.9MB/s] 


model.safetensors:   0%|          | 0.00/22.9M [00:00<?, ?B/s]


Epoch 1/100


Train Loss: 15.9984, Acc: 0.1042
Test  Loss: 7.3344, Acc: 0.1003

Epoch 2/100


Train Loss: 10.4319, Acc: 0.1083
Test  Loss: 5.7018, Acc: 0.1125

Epoch 3/100


...

Train Loss: 1.6835, Acc: 0.3941
Test  Loss: 1.7718, Acc: 0.3902

Epoch 97/100


Train Loss: 1.6523, Acc: 0.4027
Test  Loss: 1.6904, Acc: 0.3821

Epoch 98/100


Train Loss: 1.7083, Acc: 0.3824
Test  Loss: 1.7798, Acc: 0.3619

Epoch 99/100


Train Loss: 1.6715, Acc: 0.3958
Test  Loss: 1.7978, Acc: 0.3716

Epoch 100/100


Train Loss: 1.6688, Acc: 0.3992
Test  Loss: 1.5379, Acc: 0.4249

Best Accuracy on cifar10: 0.4249
